In [ ]:
from tensorflow import keras
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import load_model
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import re
import csv
import tqdm.notebook as tq
import glob
import os

## Read Dataset

In [ ]:
def read_file( file_name ):
  print('reading' , file_name )
  f =  open( file_name ,'r')
  lines = f.readlines()
  x = []
  y = []
  for line in tq.tqdm( lines ):
    line = line.rstrip()
    line = line.split(',')
    y.append( int( line[0] ) )
    x.append(  ' '.join( line[1:] )  )

  #convert y to one hot vector

  y = np.array(y,dtype = int)
  b = np.zeros( ( y.size, y.max()+1)  )
  b[np.arange(y.size),y] = 1
  y = b
  return x,y

## Change Paths Accordingly

In [ ]:
current_dir = '/content/drive/My Drive/LSTM-deneme/'
train_path = '/content/drive/My Drive/LSTM-deneme/train.csv'
test_path  = '/content/drive/My Drive/LSTM-deneme/test.csv'

In [ ]:
train_x, train_y = read_file( train_path )
test_x, test_y  = read_file( test_path  )

n_class = train_y.shape[1]

reading /content/drive/My Drive/LSTM-deneme/train.csv



reading /content/drive/My Drive/LSTM-deneme/test.csv


## Split Train and Validation Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
n_train = 500000

val_x = train_x[ n_train: ]
val_y = train_y[ n_train:, : ]

train_x = train_x[ : n_train ]
train_y = train_y[ : n_train, : ]

## Preprocessing

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_num_words = 10000

# create the tokenizer
t = Tokenizer( num_words = max_num_words )

# fit the tokenizer on the documents
t.fit_on_texts( train_x )

train_x = t.texts_to_sequences( train_x )
val_x   = t.texts_to_sequences( val_x )
test_x  = t.texts_to_sequences( test_x )

train_x = pad_sequences(  train_x )
val_x = pad_sequences(  val_x )
test_x = pad_sequences(  test_x )

## Define Model


In [ ]:
embed_dim = 128
lstm_out = 64
batch_size = 1024
lr = 0.01

model = Sequential()
model.add( Embedding( max_num_words, embed_dim, input_length = train_x.shape[1] ) )
model.add( SimpleRNN( lstm_out ) )
model.add( Dense( n_class ,activation='softmax') )

opt = keras.optimizers.Adam( learning_rate=lr )
model.compile( loss = 'categorical_crossentropy', optimizer=opt ,metrics = ['accuracy'] )
print( model.summary() )

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 34, 128)           1280000   
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 64)                12352     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                650       
Total params: 1,293,002
Trainable params: 1,293,002
Non-trainable params: 0
_________________________________________________________________
None


## Train Model !

In [ ]:
!rm -r rnn_best
checkpoint_filepath = 'rnn_best'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit( train_x, train_y, shuffle = True ,epochs=10, batch_size = batch_size , verbose=1 ,callbacks=[model_checkpoint_callback], validation_data=( val_x, val_y ) )

Epoch 1/10
489/489 [==============================] - 20s 39ms/step - loss: 1.3709 - accuracy: 0.4980 - val_loss: 1.0340 - val_accuracy: 0.6285
INFO:tensorflow:Assets written to: rnn_best/assets
Epoch 2/10
489/489 [==============================] - 19s 39ms/step - loss: 0.9495 - accuracy: 0.6578 - val_loss: 1.0054 - val_accuracy: 0.6392
INFO:tensorflow:Assets written to: rnn_best/assets
Epoch 3/10
489/489 [==============================] - 19s 38ms/step - loss: 0.8667 - accuracy: 0.6876 - val_loss: 1.0102 - val_accuracy: 0.6466
INFO:tensorflow:Assets written to: rnn_best/assets
Epoch 4/10
489/489 [==============================] - 19s 39ms/step - loss: 0.8381 - accuracy: 0.6983 - val_loss: 1.0146 - val_accuracy: 0.6367
Epoch 5/10
489/489 [==============================] - 19s 39ms/step - loss: 0.8330 - accuracy: 0.6999 - val_loss: 1.0275 - val_accuracy: 0.6389
Epoch 6/10
489/489 [==============================] - 19s 39ms/step - loss: 0.8451 - accuracy: 0.6954 - val_loss: 1.0425 - val_

In [ ]:
model = keras.models.load_model("rnn_best")

## Evalute Model

In [ ]:
results = model.evaluate( test_x, test_y, verbose = 1 )

5635/5635 [==============================] - 27s 5ms/step - loss: 1.0272 - accuracy: 0.6394
